## 7.5 NotyApp

Aplikacja będzie zawierać prostą listę notatek/zadań zapisaną w bazie danych `SQLite`. Lista będzie dostępna z poziomu `Widget` jak i w samej aplikacji. Przyjrzymy się jak zaimplemntować podstawowe elementy komunikacji między widgetem a naszą aplikacją, oraz jak aktualizować dane wyświetlane w `ListView` na `Widget`.

### **Layout**

Rozpocznijmy od utworzenia layoutów dla samego widgetu oraz dla pojedynczego elementu `ListView` (podobnie jak w `RecyclerView`). Layout widgetu będzie składał się z samego `ListView` oraz pola `TextView`, które będzie wyświetlane gdy lista jest pusta

In [ ]:
// noty_widget_provider.xml
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    style="@style/Widget.NotyJava.AppWidget.Container"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:theme="@style/Theme.NotyJava.AppWidgetContainer">

    <ListView
        android:id="@+id/listViewWidget"
        android:layout_width="match_parent"
        android:layout_height="match_parent"/>

    <TextView
        android:id="@+id/emptyViewTextView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:textSize="20sp"
        android:text="lista jest pusta"/>
</RelativeLayout>

Następnie dodajmy layout pojedynczego elementu listy - wstępnie będzie to tylko pole `TextView`

In [ ]:
// item_view.xml
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    android:layout_width="match_parent"
    android:layout_height="wrap_content">

    <TextView
        android:id="@+id/itemListTextView"
        android:layout_width="match_parent"
        android:layout_height="110dp"
        android:gravity="center"
        android:text="przykład długiego tekstu"
        android:background="@color/teal_200"
        android:textColor="@color/black"/>

</RelativeLayout>

Dodajmy opis widgetu do katalogu `xml`

In [ ]:
// noty_widget_provider_info.xml
<appwidget-provider xmlns:android="http://schemas.android.com/apk/res/android"
    android:initialLayout="@layout/noty_widget_provider"
    android:minHeight="110dp"
    android:minResizeHeight="40dp"
    android:minWidth="110dp"
    android:resizeMode="vertical|horizontal"
    android:updatePeriodMillis="36000000"
    android:widgetCategory="home_screen" />

Wstępnie dane będziemy pobierać (jak zwykle) z `DataProvider`

In [ ]:
object DataProvider {
    val dummyData = listOf(
        "notatka 1", 
        "notatka 2", 
        "notatka 3", 
        "notatka 4", 
        "notatka 5", 
        "notatka 6", 
        "notatka 7", 
        "notatka 8", 
        "notatka 9")
}

### **RemoteViewsService**

Rozpoczniemy od implementacji odpowiedniej usługi - jest ona niezbędna ponieważ widget działa na kompletnie innym procesie i nie możemy się komunikować wprost. Klasa `RemoteViewsService` jest usługą, z którą widgety będą się łączyć aby adapter mógł otrzymać instancje `RemoteViews`

In [ ]:
class NotyWidgetService : RemoteViewsService() {
    override fun onGetViewFactory(intent: Intent): RemoteViewsFactory {
        return null
    }

Musimy zaimplementować jedną metodę `onGetViewFactory`, która zwraca obiekt dostarczający dane. W tym celu zaimplementujemy klasę implementującą interfejs `RemoteViewsFactory` zawierającą szerego niezbędnych metod
- `onCreate` - tutaj łączymy się z bazą danych, wywoływana zaraz po konstruktorze
- `onDataSetChanged` - wywoływana gdy adapter wywołuje `notifyDataSetChanged`
- `onDestroy` - wywoływana w momencie usunięcia powiązania z ostatnim adapterem
- `getCount` - zwraca liczbę elementów w kolekcji
- `getViewAt` - zwraca obiekt `View` powiązany z daną pozycją
- `getLoadingView` - zezwala na załączenie własnego `View` dla ekranu ładowania
- `getViewTypeCount` - zwraca liczbę **różnych** typów `View` wykorzystywanych przez adapter - w tym przykładzie będziemy wykorzystywać jeden typ
- `getItemId` - zwraca identyfikator - tutaj będzie to odpowiednikiem rzędu na liście
- `hasStableIds` - określa czy identyfikatory pozostają niezmienne przy zmianie powiązanych z nim danych

In [ ]:
class NotyWidgetItemFactory() : RemoteViewsFactory {
    override fun onCreate() {}

    override fun onDataSetChanged() {}
    override fun onDestroy() {}

    override fun getCount(): Int {}

    override fun getViewAt(position: Int): RemoteViews {}

    override fun getLoadingView(): RemoteViews? {}

    override fun getViewTypeCount(): Int {}

    override fun getItemId(position: Int): Long {}

    override fun hasStableIds(): Boolean {}
}

Rozpocznijmy od dodania i inicjalizacji kilku pól

In [ ]:
class NotyWidgetItemFactory(private val context: Context, intent: Intent) :
    RemoteViewsFactory {
    private val appWidgetId: Int
    private lateinit var noteList: List<String>
        
    init {
            appWidgetId = intent.getIntExtra(
                AppWidgetManager.EXTRA_APPWIDGET_ID,
                AppWidgetManager.INVALID_APPWIDGET_ID)
        }  
    }

- W metodzie `onCreate` zainicjujemy listę notatek - w pierwszej fazie będzie to lista z klasy `DataProvider`

In [ ]:
override fun onCreate() {
    // otworz baze danych
    noteList = DataProvider.dummyData
}

- `getCount` zwraca wielkość listy

In [ ]:
override fun getCount(): Int = noteList.size

- W tym przykładzie będziemy wykorzystywać z domyślnego widoku ładowania, więc metoda zwraca `null`

In [ ]:
override fun getLoadingView(): RemoteViews? {
    return null
}

- Posiadamy jeden typ `View`, więc zwracamy `1`

In [ ]:
override fun getViewTypeCount(): Int = 1

- ponieważ w pierwszej fazie będziemy korzystać tylko z listy, posłużymy się pozycją jako identyfikatorem

In [ ]:
override fun getItemId(position: Int): Long = position.toLong()

- nasza kolekcja w tej chwili posiada stabilne `id`

In [ ]:
override fun hasStableIds(): Boolean = true

- w metodzie `getViewAt` zwracamy odpowiedni obiekt

In [ ]:
override fun getViewAt(position: Int): RemoteViews {
    val remoteViews = RemoteViews(context.packageName, R.layout.item_list)
    remoteViews.setTextViewText(R.id.itemListTextView, noteList[position])
    return remoteViews
}

Powracamy do klasy `NotyWidgetService` i w metodzie `onGetViewFactory` zwracamy instancję `NotyWidgetItemFactory`

In [ ]:
class NotyWidgetService : RemoteViewsService() {
    override fun onGetViewFactory(intent: Intent): RemoteViewsFactory {
        return NotyWidgetItemFactory(applicationContext, intent)
    }
    ...
}

### **AppWidgetProvider**

Drugim niezbędnym elementem będzie `AppWidgetProvider`

In [ ]:
class NotyWidgetProvider : AppWidgetProvider() {
    override fun onUpdate(
        context: Context,
        appWidgetManager: AppWidgetManager,
        appWidgetIds: IntArray
    ) {}

W metodzie `onUpdate` przechodzimy przez wszystkie instancje naszego widgetu

In [ ]:
for (appWidgetId in appWidgetIds) {

Tworzymy nowy `Intent` przekierowujący do usługi

In [ ]:
    val serviceIntent = Intent(context, NotyWidgetService::class.java)

następnie dodajmy `id` widgetu oraz wykorzystujemy metodę `setData` - metoda ta wskazuje lokalizację obiektu (przykładowo może być to plik)

In [ ]:
    serviceIntent.apply {
        putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId)
        data = Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME))
    }

Tworzymy nasze `RemoteViews`

In [ ]:
    val views = RemoteViews(context.packageName, R.layout.noty_widget_provider)

ustawiamy adapter i widok dla pustej kolekcji

In [ ]:
    views.apply {
        setRemoteAdapter(R.id.listViewWidget, serviceIntent)
        setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView)
    }

na koniec wywołujemy metodę `updateAppWidget`

In [ ]:
    appWidgetManager.updateAppWidget(appWidgetId, views)

Po wyjściu z pętli `for` wywołujemy metodę superklasy

In [ ]:
    }
super.onUpdate(context, appWidgetManager, appWidgetIds)

Pełny kod klasy `NotyWidgetProvider`

In [ ]:
class NotyWidgetProvider : AppWidgetProvider() {
    override fun onUpdate(
        context: Context,
        appWidgetManager: AppWidgetManager,
        appWidgetIds: IntArray
    ) {
        for (appWidgetId in appWidgetIds) {
            val serviceIntent = Intent(context, NotyWidgetService::class.java)
            serviceIntent.apply {
                putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId)
                data = Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME))
            }
            val views = RemoteViews(context.packageName, R.layout.noty_widget_provider)
            views.apply {
                setRemoteAdapter(R.id.listViewWidget, serviceIntent)
                setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView)
            }
            appWidgetManager.updateAppWidget(appWidgetId, views)
        }
        super.onUpdate(context, appWidgetManager, appWidgetIds)
    }
}

Do `AndroidManifest` musimy wprowadzić informacje o naszym providerze oraz service.

In [ ]:
<receiver
    android:name=".provider.NotyWidgetProvider"
    android:exported="false">
    <intent-filter>
        <action android:name="android.appwidget.action.APPWIDGET_UPDATE" />
    </intent-filter>

    <meta-data
        android:name="android.appwidget.provider"
        android:resource="@xml/noty_widget_provider_info" />
</receiver>

<service
    android:name=".service.NotyWidgetService"
    android:permission="android.permission.BIND_REMOTEVIEWS" />

Możemy przetestować aplikację i widget.

<table><tr><td><img src="https://media2.giphy.com/media/U0lFKx7L0xtZHrpaLQ/giphy.gif?cid=790b76119fc11ce6d16c4ba7009680f1447b35e164c8012a&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media2.giphy.com/media/1BNcJ99feU4EosbU19/giphy.gif?cid=790b76117e6f80ef480e50e651e2e981d067cfeb27c6e32c&rid=giphy.gif&ct=g" width="150" /></td</tr></table>

### **Odświeżanie**

Poza automatycznym odświeżaniem co 30 minut dodamy odświeżanie na przycisk umieszczony na widgecie. Zmodyfikujmy layout widgetu

In [ ]:
<RelativeLayout xmlns:android="http://schemas.android.com/apk/res/android"
    style="@style/Widget.NotyJava.AppWidget.Container"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:theme="@style/Theme.NotyJava.AppWidgetContainer">

    <LinearLayout
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:orientation="vertical">

        <Button
            android:id="@+id/refreshButtonWidget"
            android:layout_width="match_parent"
            android:backgroundTint="@color/cardview_dark_background"
            android:text="refresh"
            android:layout_height="wrap_content"/>


    <ListView
        android:id="@+id/listViewWidget"
        android:layout_width="match_parent"
        android:layout_height="match_parent" />

    </LinearLayout>

    <TextView
        android:id="@+id/emptyViewTextView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:textSize="20sp"
        android:gravity="center"
        android:text="lista jest pusta"/>
</RelativeLayout>

Przy odświeżeniu chcemy wywołać (niejawnie) metodę `onDataSetChange` klasy `NotyWidgetService`, więc przedźmy do tej metody i ją nieco zmodyfikujmy

In [ ]:
override fun onDataSetChanged() {
    DataProvider.dummyData.add(
        "Nowa notatka " + (DataProvider.dummyData.size + 1))
}

Czyli przy każdym wywołaniu tej metody będziemy dodawać nową notatkę do listy - tą funkcjonalność póżniej zmienimy. Wróćmy do klasy `NotyWidgetProvider` i obsłużmy przycisk. W pierwszej kolejności musimy utworzyć `Intent`, określimy w nim akcję którą chcemy wykonać - tutaj będzie to `ACTION_APPWIDGET_UPDATE`.

In [ ]:
val intentUpdate = Intent(context, NotyWidgetProvider::class.java)
intentUpdate.action = AppWidgetManager.ACTION_APPWIDGET_UPDATE

Następnie musimy przekazać **wszystkie** identyfikatory widgetów

In [ ]:
val idArray = intArrayOf(appWidgetId)
val intentUpdate = Intent(context, NotyWidgetProvider::class.java)
intentUpdate.apply {
    action = AppWidgetManager.ACTION_APPWIDGET_UPDATE
    putExtra(AppWidgetManager.EXTRA_APPWIDGET_IDS, idArray)
}

Tworzymy `PendingIntent` który wykona transmisję

In [ ]:
val pendingUpdate = PendingIntent.getBroadcast(
    context, appWidgetId, intentUpdate,
    PendingIntent.FLAG_UPDATE_CURRENT or PendingIntent.FLAG_MUTABLE
)

ustawiamy `setOnClickPendingIntent` na `RemoteViews`

In [ ]:
views.setOnClickPendingIntent(R.id.refreshButtonWidget, pendingUpdate)

na koniec metody `onUpdate` wykonujemy `notifyAppWidgetViewDataChanged`

In [ ]:
appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget)

Pełny kod metody `onUpdate`

In [ ]:
@RequiresApi(Build.VERSION_CODES.S)
override fun onUpdate(
    context: Context,
    appWidgetManager: AppWidgetManager,
    appWidgetIds: IntArray
) {
    for (appWidgetId in appWidgetIds) {
        val serviceIntent = Intent(context, NotyWidgetService::class.java)
        serviceIntent.apply {
            putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId)
            data = Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME))
        }
        val idArray = intArrayOf(appWidgetId)
        val intentUpdate = Intent(context, NotyWidgetProvider::class.java)
        intentUpdate.apply {
            action = AppWidgetManager.ACTION_APPWIDGET_UPDATE
            putExtra(AppWidgetManager.EXTRA_APPWIDGET_IDS, idArray)
        }

        val pendingUpdate = PendingIntent.getBroadcast(
            context, appWidgetId, intentUpdate,
            PendingIntent.FLAG_UPDATE_CURRENT or PendingIntent.FLAG_MUTABLE
        )

        val views = RemoteViews(context.packageName, R.layout.noty_widget_provider)
        views.apply {
            setRemoteAdapter(R.id.listViewWidget, serviceIntent)
            setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView)
            setOnClickPendingIntent(R.id.refreshButtonWidget, pendingUpdate)
        }
        appWidgetManager.updateAppWidget(appWidgetId, views)
        appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget)
    }
    super.onUpdate(context, appWidgetManager, appWidgetIds)
}

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/1MmSrqHNPMJ7t6sJd5/giphy.gif?cid=790b761108493c629628d2f83d3b785b8ada6d0dad9b1f6b&rid=giphy.gif&ct=g" width="150" />

### **ListView onClick**

Kolejnym krokiem będzie modyfikacja elementu listy po kliknięciu. Ponieważ tworzenie `PendingIntent` dla każdego elementu listy jest operacją niezwykle kosztochłonną wykonujemy to w dwóch krokach. W klasie `NotyWidgetProvider` utworzymy `PendingIntentTemplate` - szablon intentu. Oraz `FillIntent` przez który prześlemy potrzebne dane. W pierwszym kroku zdefiniujmy `Intent` w klasie `NotyWidgetProvider` w którym zdefiniujemy akcję.

In [ ]:
val clickIntent = Intent(context, NotyWidgetProvider::class.java)

Musimy określić identyfikator, który posłuży nam do wykonania akcji

In [ ]:
const val ACTION_DONE = "actionDone"

In [ ]:
clickIntent.action = ACTION_DONE

Następnie tworzymy `PendingIntent`

In [ ]:
val clickPendingIntent = PendingIntent.getBroadcast(
    context, 0, clickIntent,
    PendingIntent.FLAG_MUTABLE
)

oraz ustawiamy `PendingIntentTemplate`

In [ ]:
val views = RemoteViews(context.packageName, R.layout.noty_widget_provider)
views.apply {
    setRemoteAdapter(R.id.listViewWidget, serviceIntent)
    setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView)
    // ustawiamy pendingIntentTemplate
    setPendingIntentTemplate(R.id.listViewWidget, clickPendingIntent)
    setOnClickPendingIntent(R.id.refreshButtonWidget, pendingUpdate)
}

Zmodyfikowana metoda `onUpdate`

In [ ]:
const val ACTION_DONE = "actionDone"

class NotyWidgetProvider : AppWidgetProvider() {

    @RequiresApi(Build.VERSION_CODES.S)
    override fun onUpdate(
        context: Context,
        appWidgetManager: AppWidgetManager,
        appWidgetIds: IntArray
    ) {
        for (appWidgetId in appWidgetIds) {
            val serviceIntent = Intent(context, NotyWidgetService::class.java)
            serviceIntent.apply {
                putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId)
                data = Uri.parse(serviceIntent.toUri(Intent.URI_INTENT_SCHEME))
            }

            val clickIntent = Intent(context, NotyWidgetProvider::class.java)
            clickIntent.action = ACTION_DONE

            val clickPendingIntent = PendingIntent.getBroadcast(
                context, 0, clickIntent,
                PendingIntent.FLAG_MUTABLE
            )
            val idArray = intArrayOf(appWidgetId)
            val intentUpdate = Intent(context, NotyWidgetProvider::class.java)
            intentUpdate.apply {
                action = AppWidgetManager.ACTION_APPWIDGET_UPDATE
                putExtra(AppWidgetManager.EXTRA_APPWIDGET_IDS, idArray)
            }

            val pendingUpdate = PendingIntent.getBroadcast(
                context, appWidgetId, intentUpdate,
                PendingIntent.FLAG_UPDATE_CURRENT or PendingIntent.FLAG_MUTABLE
            )

            val views = RemoteViews(context.packageName, R.layout.noty_widget_provider)
            views.apply {
                setRemoteAdapter(R.id.listViewWidget, serviceIntent)
                setEmptyView(R.id.listViewWidget, R.id.emptyViewTextView)
                setPendingIntentTemplate(R.id.listViewWidget, clickPendingIntent)
                setOnClickPendingIntent(R.id.refreshButtonWidget, pendingUpdate)
            }
            appWidgetManager.updateAppWidget(appWidgetId, views)
            appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget)
        }
        super.onUpdate(context, appWidgetManager, appWidgetIds)
    }
}

Przechodzimy do klasy `NotyWidgetService`, w metodzie `getViewAt` tworzymy `FillIntent` i przesyłamy niezbędne dane

In [ ]:
val fillIntent = Intent()
fillIntent.apply {
    putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId)
    putExtra("position", position)
}
remoteViews.setOnClickFillInIntent(R.id.itemListTextView, fillIntent)

Ostatnim krokiem jest odebranie danych i wykonanie akcji, robimy to w klasie `NotyWidgetProvider` nadpisując metodę `onReceive`

In [ ]:
override fun onReceive(context: Context?, intent: Intent) {
    super.onReceive(context, intent)
}

Wpierw określmy akcję którą będziemy obsługiwać

In [ ]:
if (ACTION_DONE == intent.action) {

rozpakujmy dane

In [ ]:
val appWidgetId = intent.getIntExtra(
    AppWidgetManager.EXTRA_APPWIDGET_ID,
    AppWidgetManager.INVALID_APPWIDGET_ID
)
val position = intent.getIntExtra("position", 100)

Pobierzmy instancję `AppWidgetManager`

In [ ]:
val appWidgetManager = AppWidgetManager.getInstance(context)

i zmodyfikujmy element na liście

In [ ]:
DataProvider.dummyData[position] = "zmiana"

następnie wykonujemy `notifyAppWidgetViewDataChanged`

In [ ]:
appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget)

Pełna metoda `onReceive`

In [ ]:
override fun onReceive(context: Context?, intent: Intent) {
    if (ACTION_DONE == intent.action) {
        val appWidgetId = intent.getIntExtra(
            AppWidgetManager.EXTRA_APPWIDGET_ID,
            AppWidgetManager.INVALID_APPWIDGET_ID
        )
        val position = intent.getIntExtra("position", 100)
        val appWidgetManager = AppWidgetManager.getInstance(context)
        DataProvider.dummyData[position] = "zmiana"
        appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget)
    }
    super.onReceive(context, intent)
}

Możemy przetestować aplikację - po kliknięciu każdego elementu listy, zostanie on zmieniony ale również zostanie dodany nowy element, ponieważ będzie wywołana metoda `onDataSetChanged`

<img src="https://media2.giphy.com/media/aCLr1hpAQuVMUFKCZb/giphy.gif?cid=790b7611a7a7f667dcf133dcf9cd4bf638e9c5bf8dbc1f31&rid=giphy.gif&ct=g" width="150" />

### **Baza danych**

Notatki będziemy przechowywać w bazie danych, notatkę przeczytaną/zrealizowaną oznaczymy innym kolorem tekstu. Oprócz tego dodamy również godzinę wykonania zadania. Rozpocznijmy od zdefiniowania modelu

In [ ]:
class NoteModel(var textNote: String, val time: LocalTime) {
    var id = 0
        private set
    var color = Color.BLACK

    constructor(id: Int, textNote: String, time: LocalTime, color: Int) : this(textNote, time) {
        this.id = id
        this.color = color
    }
}

Zdefiniujmy również dane testowe w klasie `DataProvider`

In [ ]:
val notes: List<NoteModel> = listOf(
    NoteModel("notatka 1", LocalTime.of(12, 0)),
    NoteModel("notatka 2", LocalTime.of(13, 0)),
    NoteModel("notatka 3", LocalTime.of(21, 0)),
    NoteModel("notatka 4", LocalTime.of(9, 9)),
    NoteModel("notatka 5", LocalTime.of(22, 34)),
    NoteModel("notatka 6", LocalTime.of(11, 22)),
    NoteModel("notatka 7", localTime)
)

Stwórzmy naszą bazę danych

In [ ]:
class DBHandler(context: Context) :
    SQLiteOpenHelper(context, DATABASE_NAME, null, DATABASE_VERSION) {
    override fun onCreate(db: SQLiteDatabase) {
        val CREATE_STUDENTS_TABLE = "CREATE TABLE " +
                NOTES_TABLE +
                "(" +
                COLUMN_ID + " " +
                "INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL," +
                COLUMN_TEXT +
                " TEXT," +
                COLUMN_TIME +
                " TEXT," +
                COLUMN_COLOR +
                " INTEGER" +
                ")"
        db.execSQL(CREATE_STUDENTS_TABLE)
    }

    override fun onUpgrade(db: SQLiteDatabase, oldVersion: Int, newVersion: Int) {
        db.execSQL("DROP TABLE IF EXISTS $NOTES_TABLE")
        onCreate(db)
    }

    companion object {
        private const val DATABASE_VERSION = 1
        private const val DATABASE_NAME = "notesBDo.db"
        private const val NOTES_TABLE = "NotesTable"

        private const val COLUMN_ID = "_id"
        private const val COLUMN_TEXT = "text"
        private const val COLUMN_TIME = "time"
        private const val COLUMN_COLOR = "color"
    }
}

Będziemy potrzebować kilka metod, zacznijmy od dodania nowego wpisu do bazy

In [ ]:
    fun addNote(note: NoteModel) {
        val db = this.writableDatabase
        val values = ContentValues()
        values.apply {
            put(COLUMN_TEXT, note.textNote)
            put(COLUMN_TIME, note.time.toString())
            put(COLUMN_COLOR, note.color)
        }
        db.insert(NOTES_TABLE, null, values)
        db.close()
    }

Przejdźmy do metody zwracającą listę wszystkich notatek

In [ ]:
    val notes: List<NoteModel>
        get() {
            val notes = mutableListOf<NoteModel>()
            val db = this.readableDatabase
            val cursor = db.rawQuery("SELECT * FROM $NOTES_TABLE", null)
            if (cursor.moveToFirst()) {
                do {
                    notes.add(
                        NoteModel(
                            cursor.getInt(0),
                            cursor.getString(1),
                            LocalTime.parse(cursor.getString(2)),
                            cursor.getInt(3)
                        )
                    )
                } while (cursor.moveToNext())
            }
            db.close()
            cursor.close()
            return notes
        }

Będziemy również aktualizować kolor danej notatki

In [ ]:
    fun updateNote(id: Int) {
        val db = this.writableDatabase
        val contentValues = ContentValues()
        contentValues.put(COLUMN_COLOR, Color.CYAN)
        db.update(
            NOTES_TABLE,
            contentValues,
            "$COLUMN_ID=$id",
            null
        )
        db.close()
    }

Zainicjujmy naszą bazę za pomocą danych testowych w klasie `MainActivity`

In [ ]:
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContentView(R.layout.activity_main)
        val dbHandler = DBHandler(this)
        DataProvider.notes.forEach { dbHandler.addNote(it) }
        dbHandler.close()
    }

Dokonajmy zmian w klasie `NotyWidgetItemFactory`, będziemy potrzebować zmienną `DBHandler`

In [ ]:
private DBHandler dbHandler;

Bazę zainicjujemy w metodzie `onCreate`, tutaj również zainicjujemy lokalną listę

In [ ]:
    override fun onCreate() {
        dbHandler = DBHandler(context)
        noteList = dbHandler.notes
    }

Przy każdej zmianie chcemy odświeżyć listę i pobrać aktualne dane

In [ ]:
    override fun onDataSetChanged() {
        noteList = dbHandler.notes
    }

W metodzie `onDestroy` zamykamy dostęp do bazy

In [ ]:
    override fun onDestroy() {
        dbHandler.close()
    }

W metodzie `getViewAt` ustawmy tekst oraz kolor tekstu elementów listy

In [ ]:
        val remoteViews = RemoteViews(
            context.packageName, 
            R.layout.item_list)
        remoteViews.setTextViewText(
            R.id.itemListTextView, 
            noteList[position].time.toString() + 
                "\n" + noteList[position].textNote
        )
        remoteViews.setTextColor(
            R.id.itemListTextView, 
            noteList[position].color)

Przez `FillIntent` będziemy przesyłać `appWidgetId` oraz `id` elementu listy

In [ ]:
        val fillIntent = Intent()
        fillIntent.apply {
            putExtra(AppWidgetManager.EXTRA_APPWIDGET_ID, appWidgetId)
            putExtra("id", noteList[position].id)
        }
        remoteViews.setOnClickFillInIntent(
            R.id.itemListTextView, 
            fillIntent)
        return remoteViews

Zmienimy również `getItemId` - teraz będziemy posługiwać się `id` z bazy danych

In [ ]:
override fun getItemId(position: Int): Long = noteList[position].id.toLong()

Przejdźmy do klasy `NotyWidgetProvider` i w metodzie `onReceive` zaktualizujmy element listy

In [ ]:
    override fun onReceive(context: Context, intent: Intent) {
        if (ACTION_DONE == intent.action) {
            val appWidgetId = intent.getIntExtra(
                AppWidgetManager.EXTRA_APPWIDGET_ID,
                AppWidgetManager.INVALID_APPWIDGET_ID
            )
            val id = intent.getIntExtra("id", 100)
            val appWidgetManager = AppWidgetManager.getInstance(context)
            val dbHandler = DBHandler(context)
            dbHandler.updateNote(id)
            dbHandler.close()
            appWidgetManager.notifyAppWidgetViewDataChanged(appWidgetId, R.id.listViewWidget)
        }
        super.onReceive(context, intent)
    }

Możemy przetestować aplikację

<img src="https://media2.giphy.com/media/jCI4kz73h3KJEs0AJj/giphy.gif" width="150" />